In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re
import string
from textblob import TextBlob
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
# read all files 
files = sorted(glob.glob("../data/raw/*_articles.csv"))

file_names = []

for file_path in files:
    file_name = re.split("_", os.path.basename(file_path))[1].split('.')[0]
    
    hd = pd.read_csv(file_path)
    file_names.append(hd)
    
df = pd.concat(file_names).reset_index().drop(["Unnamed: 0", "index"], axis=1)

In [5]:
# read metadata file
newsrooms = pd.read_csv('../data/raw/newsrooms.csv')
newsrooms.columns = ['site', 'monthly_visits', 'country', 'country_of_pub']
newsrooms.dropna(inplace=True)
newsrooms = newsrooms.drop_duplicates(subset = 'site')
newsrooms = newsrooms.sort_values(by=['country_of_pub', 'monthly_visits'], ascending=False).reset_index().drop('index', axis=1)

In [6]:
df = pd.merge(df, newsrooms)

In [7]:
print("This dataset has", len(df[df['country_of_pub']=='South Africa']), "headlines from South Africa", len(df[df['country_of_pub']=='India']), "headlines from India",
      len(df[df['country_of_pub']=='UK']), "headlines from the UK and", len(df[df['country_of_pub']=='USA']), "headlines from the U.S.")

This dataset has 7814 headlines from South Africa 6779 headlines from India 11210 headlines from the UK and 22376 headlines from the U.S.


In [8]:
print("This dataset covers", len(df.site[df['country_of_pub']=='South Africa'].unique()), "news outlets from South Africa", len(df.site[df['country_of_pub']=='India'].unique()), 
      "news outlets from India", len(df.site[df['country_of_pub']=='UK'].unique()), "news outlets from the UK and", len(df.site[df['country_of_pub']=='USA'].unique()), "news outlets from the U.S.")

This dataset covers 29 news outlets from South Africa 24 news outlets from India 43 news outlets from the UK and 90 news outlets from the U.S.


In [149]:
# emoticons_happy = set([
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     ])

# # Sad Emoticons
# emoticons_sad = set([
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
#     ])

# #Emoji patterns
# emoji_pattern = re.compile("["
#          u"\U0001F600-\U0001F64F"  # emoticons
#          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#          u"\U0001F680-\U0001F6FF"  # transport & map symbols
#          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#          u"\U00002702-\U000027B0"
#          u"\U000024C2-\U0001F251"
#          "]+", flags=re.UNICODE)

# #combine sad and happy emoticons
# emoticons = emoticons_happy.union(emoticons_sad)

In [167]:
# def clean_tweets(tweet):
 
#     stop_words = set(stopwords.words('english'))
#     word_tokens = word_tokenize(tweet)
#     #after tweepy preprocessing the colon symbol left remain after      #removing mentions
#     tweet = re.sub(r':', '', tweet)
#     tweet = re.sub(r'‚Ä¶', '', tweet)
#     tweet = tweet.replace("'s", '')
#     #replace consecutive non-ASCII characters with a space
#     tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#     #remove emojis from tweet
#     tweet = emoji_pattern.sub(r'', tweet)
#     tweet = tweet.lower()
#     #filter using NLTK library append it to a string
#     filtered_tweet = [w for w in word_tokens if not w in stop_words]
#     filtered_tweet = []
#     #looping through conditions
#     for w in word_tokens:
#     #check tokens against stop words , emoticons and punctuations
#         if w not in stop_words and w not in emoticons and w not in string.punctuation:
#             filtered_tweet.append(w)
#     return ' '.join(filtered_tweet)
#     #print(word_tokens)
#     #print(filtered_sentence)return tweet
    

In [169]:
# %%time
# def remove_pattern(input_txt, pattern):
#     r = re.findall(pattern, input_txt)
#     for i in r:
#         input_txt = re.sub(i, '', input_txt)        
#     return input_txt
# def vec_tweets(tweets):
#     #remove twitter Return handles (RT @xxx:)
#     tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
#     #remove twitter handles (@xxx)
#     tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
#     #remove URL links (httpxxx)
#     tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")

#     #remove special characters, numbers, punctuations (except for #)
#     tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
#     return tweets

# # tweets_df['clean_text']= clean_tweets(tweets_df['clean_text'].astype('str')) #The function clean_tweets were put to use.
# # tweets_df.tail()

Wall time: 0 ns


In [424]:
# from https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api

nlp = spacy.load('en_core_web_lg')

# Choose model accordingly for contractions function
# model = api.load("glove-twitter-25")
# # model = api.load("glove-twitter-100")
# model = api.load("word2vec-google-news-300")

# cont = Contractions(kv_model=model)
# cont.load_models()


# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


# def expand_contractions(text):
#     """expand shortened words, e.g. don't to do not"""
#     text = list(cont.expand_texts([text], precise=True))[0]
#     return text


def text_preprocessing(text, accented_chars=True, contractions=False, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [425]:
%%time
df['headline_no_site'] = df['headline'].apply(lambda x: ''.join(re.split('(?<=[a-z0-9])[A-Z]', x)[:-1]))
df['clean_headline'] = df['headline_no_site'].apply(text_preprocessing)
df['subtitle'][df['subtitle'].isna() == True] = ""
df['clean_subtitle'] = df['subtitle'].apply(text_preprocessing)

C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 13min 59s


In [427]:
df.to_csv('../data/processed/headlines_cl.csv')

In [9]:
df_cl = pd.read_csv('../data/processed/headlines_cl.csv')

In [180]:
%%time
df['clean_headline'] = df['headline'].apply(clean_tweets).apply(vec_tweets)
df['subtitle'][df['subtitle'].isna() == True] = ""
df['clean_subtitle'] = df['subtitle'].apply(clean_tweets).apply(vec_tweets)

C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Wall time: 17.8 s


In [188]:
# translate time to english
from googletrans import Translator
translator = Translator()

df['clean_subtitle'] = translator.translate('9 ore fa', src='it', dest='en')

In [186]:
translated.text

'9 hours ago'

In [153]:
# list(df['headline'])

In [ ]:
%%time
#importing and initialising the VADER analyser
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#Storing the scores in list of dictionaries
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(tweets_df['clean_text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['Tweet'][i]))
    compound = analyzer.polarity_scores(tweets_df['clean_text'][i])["compound"]
    pos = analyzer.polarity_scores(tweets_df['clean_text'][i])["pos"]
    neu = analyzer.polarity_scores(tweets_df['clean_text'][i])["neu"]
    neg = analyzer.polarity_scores(tweets_df['clean_text'][i])["neg"]
    
    scores.append({"compound_s": compound,
                       "positive_s": pos,
                       "negative_s": neg,
                       "neutral_s": neu
                  })

#Appending the scores into the dataframe for further analysis 
sentiments_score = pd.DataFrame.from_dict(scores)
tweets_df_sent = tweets_df.join(sentiments_score)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.hist(tweets_df_sent['compound_s'].values, bins=40)
plt.title('Histogram of Compound Sentiment')
plt.ylabel('Number of Tweets')
plt.xlabel('Compound Sentiment')